In [60]:
import pandas as pd
import numpy as np
import re,json
from collections import defaultdict,Counter

from sklearn.feature_extraction.text import TfidfVectorizer

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### 테스트 데이터 load

In [2]:

tmp_data = [[10,'갤럭시 중고폰'],[11,'연예인 지갑 지갑'],[12,'갤럭시s10 갤럭시북 NT950XDZ-G58AW 중고폰']]
tmp = pd.DataFrame(tmp_data, columns=['id','name'])

#tmp=tmp.set_index('id')
tmp.head()

,id,name
0,10,갤럭시 중고폰
1,11,연예인 지갑 지갑
2,12,갤럭시s10 갤럭시북 NT950XDZ-G58AW 중고폰


## 1. 토큰화

In [3]:
def tokenizer(data):
    #token=[]
    token = ''

    data = data.lower() #소문자로 변환

    words = data.split() #공백으로 분리
    #print("split ", words)

    #규칙에 해당 -> findall
    '''
    - 공백 기준으로 분리
    - 영어는 소문자로 변환
    - 공백으로 분리된 텍스트 안에서 다음 규칙으로 토큰을 구분
    - 연속된 한글 : [가-힣]+
    - 연속된 자모 : [ㄱ-ㅎ|ㅏ-ㅣ]+
    - 연속된 영문, 숫자, 하이픈(-) : [a-zA-Z0-9-]+
    - 그 외 문자는 묶어서 하나로 취급 : [^ A-Za-z0-9가-힣+] (연속된 특수문자)
    '''
    p = re.compile("[가-힣]+|[ㄱ-ㅎ|ㅏ-ㅣ]+|[0-9a-z]+|[a-z0-9-]+|[^ a-z0-9가-힣+]") #규칙

    for word in words:
        find = re.findall(p,word)
        for w in find:
            token=token+w+'\n'
            #token.append(w)
    return token

In [4]:
tmp['token']=tmp['name'].apply(tokenizer)
tmp.head()

,id,name,token
0,10,갤럭시 중고폰,갤럭시\n중고폰\n
1,11,연예인 지갑 지갑,연예인\n지갑\n지갑\n
2,12,갤럭시s10 갤럭시북 NT950XDZ-G58AW 중고폰,갤럭시\ns10\n갤럭시북\nnt950xdz\n-g58aw\n중고폰\n


## 2.inverted index

In [5]:
js = tmp.to_json(orient = 'records')
json_data =json.loads(js)
json_data

[{'id': 10, 'name': '갤럭시 중고폰', 'token': '갤럭시\n중고폰\n'},
 {'id': 11, 'name': '연예인 지갑 지갑', 'token': '연예인\n지갑\n지갑\n'},
 {'id': 12,
  'name': '갤럭시s10 갤럭시북 NT950XDZ-G58AW 중고폰',
  'token': '갤럭시\ns10\n갤럭시북\nnt950xdz\n-g58aw\n중고폰\n'}]

In [22]:

index_dict=defaultdict(list)

for data in json_data:
    for token in data['token'].split():
        if token in data['token']:
            if token in index_dict:
                index_dict[token].append(data['id'])
            else:
                index_dict[token]=[data['id']]
                
print(index_dict)

defaultdict(<class 'list'>, {'갤럭시': [10, 12], '중고폰': [10, 12], '연예인': [11], '지갑': [11, 11], 's10': [12], '갤럭시북': [12], 'nt950xdz': [12], '-g58aw': [12]})


## 3.tf-idf 적용 (TfidfVectorizer )

In [8]:
## token 많아질수록 벡터의 차원이 커지는 문제
## 가장 많이 나온 단어 n개만 사용하는 max_features 파라미터 : TfidfVectorizer(max_features=4)
## all_teokn = tmp['token ] : list

words=tmp['token']
#print(words)

vect2 = TfidfVectorizer()
tfvect_matrix = vect2.fit_transform(words)
tfvect_matrix.toarray()

array([[0.        , 0.        , 0.        , 0.70710678, 0.        ,
        0.        , 0.70710678, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.4472136 , 0.        , 0.89442719],
       [0.44036207, 0.44036207, 0.44036207, 0.3349067 , 0.44036207,
        0.        , 0.3349067 , 0.        ]])

In [9]:
token_tf = vect2.get_feature_names() #유니크한 단어수
token_tf

['g58aw', 'nt950xdz', 's10', '갤럭시', '갤럭시북', '연예인', '중고폰', '지갑']

In [10]:
## tf-idf적용한 결과 df로 생성 

tfidv_df = pd.DataFrame(tfvect_matrix.toarray(), columns = sorted(token_tf))
tfidv_df.index = list(tmp['id'])
tfidv_df

,g58aw,nt950xdz,s10,갤럭시,갤럭시북,연예인,중고폰,지갑
10,0.000000,0.000000,0.000000,0.707107,0.000000,0.000000,0.707107,0.000000
11,0.000000,0.000000,0.000000,0.000000,0.000000,0.447214,0.000000,0.894427
12,0.440362,0.440362,0.440362,0.334907,0.440362,0.000000,0.334907,0.000000


## 4. 검색어(query) 테스트

- 입력한 키워드 토큰화 리스트 : q_token
- reverted_index로 입력한 quey가 있는 문서id 리스트 리턴  
- 문서들의 교집합을 찾음 : q_documents 
<br><br>
- 교집합 문서들의 name : search_dc
- 교집합 문서들의 td-idf(score) : search_tf
- search_dc와 search_tf merge : search
- score기준 내림차순 정렬 
<br><br>
- 최종결과 response message

In [29]:
q = "갤럭시 중고폰"

#입력한 키워드 토큰화 
q_token = tokenizer(q).split()
#print(q_token)

# query가 들어있는 문서id
q_documents=[]
for tk in q_token:
    q_documents.append(set(index_dict[tk])) 
#print(q_documents)

# 문서들의 교집합 
query_documents = list(q_documents[0].intersection(*q_documents))
print(query_documents)

[10, 12]


In [80]:
#교집합 문서들의 id,name
search_dc = tmp[tmp['id'].isin(query_documents)]
search_dc=search_dc.set_index('id')
search_dc

#교집합 문서들에 대해서 tf-dif(score값)
#tfidv_df.loc[query_documents] #교집합문서
search_tf = tfidv_df.loc[query_documents][q_token] 

search_tf['score'] = search_tf.sum(axis=1)
search_tf

# search_dc와 search_tf joiin (by id)
search=search_tf.join(search_dc,how='inner')
search['pid']=search.index
search.sort_values(by=['score'],ascending=[False],inplace=True) #score기준 정렬
search

,name,token
id,,
10,갤럭시 중고폰,갤럭시\n중고폰\n
12,갤럭시s10 갤럭시북 NT950XDZ-G58AW 중고폰,갤럭시\ns10\n갤럭시북\nnt950xdz\n-g58aw\n중고폰\n


,갤럭시,중고폰,score
10,0.707107,0.707107,1.414214
12,0.334907,0.334907,0.669813


,갤럭시,중고폰,score,name,token,pid
10,0.707107,0.707107,1.414214,갤럭시 중고폰,갤럭시\n중고폰\n,10
12,0.334907,0.334907,0.669813,갤럭시s10 갤럭시북 NT950XDZ-G58AW 중고폰,갤럭시\ns10\n갤럭시북\nnt950xdz\n-g58aw\n중고폰\n,12


In [81]:
# 최종결과 
response = search[['pid','name','score']]
response

# response msg
js = response.to_json(orient='records')
res_data =json.loads(js)
res_data

,pid,name,score
10,10,갤럭시 중고폰,1.414214
12,12,갤럭시s10 갤럭시북 NT950XDZ-G58AW 중고폰,0.669813


[{'pid': 10, 'name': '갤럭시 중고폰', 'score': 1.4142135624},
 {'pid': 12, 'name': '갤럭시s10 갤럭시북 NT950XDZ-G58AW 중고폰', 'score': 0.6698134053}]